# Full Data Analysis & Visualisation Pipeline
This notebook covers Deliverables 1 to 5: Data Preparation, Numerical Analysis, Visualisation, Database Integration, and Python Data Analysis.

In [ ]:
# 1. Data Preparation
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import numpy as np

# Load datasets
edu_df = pd.read_csv('education.csv')
labour_df = pd.read_csv('labour.csv')

# Initial inspection
print('--- Education Data Info ---')
edu_df.info()
print('
--- Labour Data Info ---')
labour_df.info()

# Filter for South Africa
edu_sa = edu_df[edu_df['REF_AREA_LABEL'] == 'South Africa'].copy()
labour_sa = labour_df[labour_df['REF_AREA_LABEL'] == 'South Africa'].copy()
print(f'
Found {len(edu_sa)} rows for South Africa in the education dataset.')
print(f'Found {len(labour_sa)} rows for South Africa in the labour dataset.')

# Select relevant columns and rename
columns_to_keep = {
    'TIME_PERIOD': 'Year',
    'INDICATOR_LABEL': 'Indicator',
    'SEX_LABEL': 'Sex',
    'AGE_LABEL': 'Age',
    'OBS_VALUE': 'Value'
}
edu_clean = edu_sa[list(columns_to_keep.keys())].rename(columns=columns_to_keep)
labour_clean = labour_sa[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

# Handle missing values
print('
--- Checking for missing values in cleaned Education data ---')
print(edu_clean.isnull().sum())
print('
--- Checking for missing values in cleaned Labour data ---')
print(labour_clean.isnull().sum())

# Descriptive statistics
print('
--- Descriptive Statistics for Education Data (South Africa) ---')
print(edu_clean.describe())
print('
--- Descriptive Statistics for Labour Data (South Africa) ---')
print(labour_clean.describe())

In [ ]:
# 2. Numerical Analysis
edu_stats = edu_clean['Value'].describe()
edu_mean = edu_clean['Value'].mean()
edu_median = edu_clean['Value'].median()
edu_mode = edu_clean['Value'].mode()[0]
edu_std = edu_clean['Value'].std()
labour_stats = labour_clean['Value'].describe()
labour_mean = labour_clean['Value'].mean()
labour_median = labour_clean['Value'].median()
labour_mode = labour_clean['Value'].mode()[0]
labour_std = labour_clean['Value'].std()
print('Education Stats:', edu_stats)
print('Mean:', edu_mean, 'Median:', edu_median, 'Mode:', edu_mode, 'Std:', edu_std)
print('Labour Stats:', labour_stats)
print('Mean:', labour_mean, 'Median:', labour_median, 'Mode:', labour_mode, 'Std:', labour_std)
# Grouped analysis
edu_by_sex = edu_clean.groupby('Sex')['Value'].describe()
labour_by_sex = labour_clean.groupby('Sex')['Value'].describe()
edu_by_age = edu_clean.groupby('Age')['Value'].describe()
labour_by_age = labour_clean.groupby('Age')['Value'].describe()
print('Education by Sex:', edu_by_sex)
print('Labour by Sex:', labour_by_sex)
print('Education by Age:', edu_by_age)
print('Labour by Age:', labour_by_age)
# Correlation analysis
edu_year = edu_clean.groupby('Year')['Value'].mean().reset_index().rename(columns={'Value': 'Education_Value'})
labour_year = labour_clean.groupby('Year')['Value'].mean().reset_index().rename(columns={'Value': 'Labour_Value'})
merged = pd.merge(edu_year, labour_year, on='Year')
correlation = merged['Education_Value'].corr(merged['Labour_Value'])
print('Correlation between Education and Labour (by Year):', correlation)
# Regression and hypothesis testing
from scipy.stats import linregress, ttest_ind
result = linregress(merged['Education_Value'], merged['Labour_Value'])
print('Regression Results:')
print(f'Slope: {result.slope}')
print(f'Intercept: {result.intercept}')
print(f'R-squared: {result.rvalue**2}')
print(f'P-value: {result.pvalue}')
print(f'Standard Error: {result.stderr}')
group1 = edu_clean[edu_clean['Age'] == '15-24']['Value']
group2 = edu_clean[edu_clean['Age'] == '25-34']['Value']
t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
print('T-test between Age 15-24 and 25-34 (Education Values):')
print(f'T-statistic: {t_stat}')
print(f'P-value: {p_val}')

In [ ]:
# 3. Visualisation
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(edu_clean['Value'], kde=True)
plt.title('Distribution of Education Attainment (%)')
plt.xlabel('Percentage')
plt.ylabel('Frequency')
plt.subplot(1, 2, 2)
sns.histplot(labour_clean['Value'], kde=True)
plt.title('Distribution of Labour Force Participation (%)')
plt.xlabel('Percentage')
plt.tight_layout()
plt.show()
plt.figure(figsize=(8, 5))
sns.scatterplot(x='Education_Value', y='Labour_Value', data=merged)
plt.title('Education vs Labour (Yearly Average)')
plt.xlabel('Education Value (%)')
plt.ylabel('Labour Value (%)')
plt.show()
plt.figure(figsize=(8, 5))
sns.boxplot(x='Sex', y='Value', data=edu_clean)
plt.title('Education Attainment by Sex')
plt.xlabel('Sex')
plt.ylabel('Education Value (%)')
plt.show()
plt.figure(figsize=(8, 5))
edu_by_age_plot = edu_clean.groupby('Age')['Value'].mean().reset_index()
sns.barplot(x='Age', y='Value', data=edu_by_age_plot)
plt.title('Average Education Attainment by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Average Education Value (%)')
plt.show()
plt.figure(figsize=(10, 5))
labour_by_year = labour_clean.groupby('Year')['Value'].mean().reset_index()
plt.plot(labour_by_year['Year'], labour_by_year['Value'], marker='o')
plt.title('Labour Force Participation Over Time')
plt.xlabel('Year')
plt.ylabel('Labour Force Participation (%)')
plt.grid(True)
plt.show()
plt.figure(figsize=(6, 6))
sex_counts = edu_clean['Sex'].value_counts()
plt.pie(sex_counts, labels=sex_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Sex in Education Data')
plt.show()

In [ ]:
# 4. Database Integration
edu_clean.to_csv('edu_clean.csv', index=False)
labour_clean.to_csv('labour_clean.csv', index=False)
merged.to_csv('merged.csv', index=False)
print('Cleaned CSV files exported: edu_clean.csv, labour_clean.csv, merged.csv')
conn = sqlite3.connect('assignment_data.db')
cursor = conn.cursor()
edu_clean.to_sql('education_clean', conn, if_exists='replace', index=False)
labour_clean.to_sql('labour_clean', conn, if_exists='replace', index=False)
merged.to_sql('merged', conn, if_exists='replace', index=False)
edu_query = pd.read_sql_query('SELECT * FROM education_clean LIMIT 5', conn)
labour_query = pd.read_sql_query('SELECT * FROM labour_clean LIMIT 5', conn)
merged_query = pd.read_sql_query('SELECT * FROM merged LIMIT 5', conn)
print('Education Clean sample:')
print(edu_query)
print('Labour Clean sample:')
print(labour_query)
print('Merged sample:')
print(merged_query)
cursor.execute('UPDATE education_clean SET Value = ? WHERE rowid = 1', (100,))
cursor.execute('DELETE FROM education_clean WHERE rowid = (SELECT MAX(rowid) FROM education_clean)')
conn.commit()
edu_updated = pd.read_sql_query('SELECT * FROM education_clean LIMIT 5', conn)
print('Updated Education Clean sample:')
print(edu_updated)
conn.close()

In [ ]:
# 5. Python Data Analysis
edu_clean = pd.read_csv('edu_clean.csv')
labour_clean = pd.read_csv('labour_clean.csv')
edu_clean['High_Attainment'] = edu_clean['Value'] > 80
Q1 = labour_clean['Value'].quantile(0.25)
Q3 = labour_clean['Value'].quantile(0.75)
IQR = Q3 - Q1
labour_clean['Outlier'] = ((labour_clean['Value'] < (Q1 - 1.5 * IQR)) | (labour_clean['Value'] > (Q3 + 1.5 * IQR)))
high_attainment_by_age = edu_clean.groupby('Age')['High_Attainment'].sum().reset_index()
plt.figure(figsize=(8,5))
plt.bar(high_attainment_by_age['Age'], high_attainment_by_age['High_Attainment'])
plt.title('Count of High Education Attainment (>80%) by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.show()
plt.figure(figsize=(10,5))
plt.plot(labour_clean['Year'], labour_clean['Value'], label='Labour Participation')
plt.scatter(labour_clean[labour_clean['Outlier']]['Year'], labour_clean[labour_clean['Outlier']]['Value'], color='red', label='Outliers')
plt.title('Labour Force Participation Over Time (Outliers Highlighted)')
plt.xlabel('Year')
plt.ylabel('Labour Force Participation (%)')
plt.legend()
plt.show()
attainment_counts = edu_clean['High_Attainment'].value_counts()
plt.figure(figsize=(6,6))
plt.pie(attainment_counts, labels=['Low/Normal', 'High'], autopct='%1.1f%%', startangle=140)
plt.title('Proportion of High vs. Low/Normal Education Attainment')
plt.show()
high_attainment_pct = 100 * attainment_counts[True] / attainment_counts.sum() if True in attainment_counts else 0
outlier_count = labour_clean['Outlier'].sum()
print(f'High education attainment (>80%) observed in {high_attainment_pct:.1f}% of records.')
print(f'Number of outlier labour participation records: {outlier_count}')

## Summary of Python Data Analysis
- Data was cleaned and transformed to flag high attainment and outliers.
- Conditional formatting highlighted key trends and anomalies.
- Charts visualised age group attainment, labour participation trends, and attainment proportions.
- Findings: High education attainment is present in a notable portion of the data, and some labour participation records are statistical outliers.